In [ ]:
import folium, json, requests, math
import pandas as pd
import numpy as np

### Create map of both stations and the influence area
the influence area has been obtained by the (rawBN) code below

In [ ]:
dfsta=pd.read_csv('../data/staions_influence_4.csv', index_col=0)
dfsta.head(2)

In [ ]:
len(dfsta),dfsta.head(1)

In [ ]:
dfsta = dfsta.rename(columns = {"CODGEO":"code"})

In [ ]:
poly=pd.read_csv('../data/communes_geometry.csv', index_col=0)
poly.head(2)

In [ ]:
final_df = dfsta.merge(poly, on = "code",  how='inner')

Convert polygons to plot them on a choropleth map

In [ ]:
from shapely import wkt

from shapely.geometry import shape
import geopandas as gpd

final_df.crs = 'epsg:4326'
final_df['geometry'] = final_df['geometry'].apply(wkt.loads)
final_df = gpd.GeoDataFrame(final_df, crs='epsg:4326')

In [ ]:
final_df.dtypes

In [ ]:
final_df.head(1)

In [ ]:
final_df.isnull().sum()

### Build MAP

In [ ]:
# Make an empty map
map = folium.Map(location = [46.71109,1.7191036],tiles='cartodbpositron',
               zoom_start=6,min_zoom = 5,max_zoom = 12)

In [ ]:
#Add title


loc = 'Air quality measurement stations'
sub = '& their 4 kms radius influenced communes'
title_html = '''
<h3 align="center" style="font-size:16px ; color:#666666; padding:5px 0px;margin:0px; z-index:10">
{}</h3>
<h4 align="center" style="font-size:12px ; color:#777777; padding:0px 0px 9px 0px;margin:0px">{}</h4>'''.format(loc,sub)   


map.get_root().html.add_child(folium.Element(title_html))



In [ ]:
style = {'fillColor': '#f5f5f5', 'lineColor': '#ffffbf'}

folium.GeoJson(
    data=final_df["geometry"],
    style_function=lambda feature: {
        'fillColor': '#cc6666',
        'color': '#993333',
        'weight': 0.3,
    }
).add_to(map)

In [ ]:
dfsta=final_df.groupby('Station').median().reset_index()
dfsta.head(1)
len(final_df),len(dfsta)

In [ ]:
coloralpha='rgba(0,0,0,0.3)'
for lat,lan,Station in zip(dfsta['lat'],dfsta['lon'],dfsta['Station']):
    # Marker() takes location coordinates
    # as a list as an argument
    folium.CircleMarker(location=[lat,lan],
                  popup = 'Station: {}.format(Station)',
                  radius=2, fill=True,
                  color=coloralpha, fill_color='#000033', opacity=0.2,fill_opacity=0.9, weight= 1,
                 ).add_to(map)
    
map

In [ ]:
map.save('stations_influence.html')

<br><br><br><br><br><br><br>
### Previous code used to obtain the 4 kms influence from each station.
Long loop